# Laboratorio Redes Neuronales

Librerias a utilizar

In [22]:
import numpy as np
from functools import reduce
import mnist_reader
from scipy import optimize as op

Metodo para convertir un array plano de thetas en matrices con sus shapes correspondientes

In [23]:
def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)

    for i in range(layers - 1):
        steps[i + 1] = steps[i] + sizes[i]

    return [
        flat_thetas[steps[i]: steps[i + 1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

Metodo feed_forward Matricial para obtener las salidas de cada capa de neuronas de cada una de las 60000 pruebas

In [24]:
def feed_forward(thetas, X):
    a = [X]

    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul(
                    np.hstack((
                        np.ones(len(X)).reshape(len(X), 1),
                        a[i]
                    )), thetas[i].T
                )
            )            
        )
    return a

Funcion de costo para la optimizacion

In [25]:
def cost_function(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

Metodo back propagation para obtener el gradiente

In [26]:
def back_propagation(flat_thetas, shapes, X, Y):
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
    
    # Paso 2.2
    a = feed_forward(thetas, X)

    # Paso 2.3
    deltas = [*range(layers - 1), a[-1] - Y]
    
    # Paso 2.4
    for i in range(layers - 2, 0, -1):
        deltas[i] = np.matmul((deltas[i + 1], np.delete(thetas[i], 0, 1))) * (a[i] * (1 - a[i]))

    # Paso 2.5 y 3
    Deltas = []
    for i in range(layers - 1):
        Deltas.append(
            ( 
            np.matmul(
                deltas[i + 1].T,
                np.hstack((
                    np.ones(len(a[i])).reshape(len(a[i]), 1),
                    a[i]))
                    )
            ) / m
        )
    Deltas = np.asarray(Deltas)

    return flatten_list_of_arrays(
        Deltas
    )

Funcion Simboide para pasar las entradas de cada capa a las salidas

In [27]:
sigmoid = lambda x: 1.0/(1.0 + np.exp(-x))

Metodo para aplanar matrices y dejarlas en un array

In [28]:
flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

# Entrenamiento

Gracias al metodo de load_mnist obtenemos directamente 60000 datos de entrenamiento con sus 784 pixeles cada uno

Ademas de tener los datos de prueba que en este caso son 10000

In [30]:
# Datos de entrenamiento 60000
X, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
# Datos de prueba 10000
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')
# Reajuste de los datos
X = X/1000.0
# Obtencion de m
m,_ = X.shape
# Remodelado de las respuestas para poder comparar con el resultado obtenido
y_train = y_train.reshape(m,1)
Y = (y_train == np.array(range(10))).astype(int)

Se establece la cantidad de neuronas y de capas.
En este caso seran 784 en la capa de entrada, 125 en la capa oculta y 10 en la capa de salida.

In [31]:
# Creacion de red neuronal con 3 capas
NEURAL_NET = np.array([
    784,
    125,
    10
])
# Shapes de las matrices de transicion
theta_shapes = np.hstack((
    NEURAL_NET[1:].reshape(len(NEURAL_NET) - 1, 1),
    (NEURAL_NET[:-1] + 1).reshape(len(NEURAL_NET) - 1, 1)
))
# Creacion de matrices de transicion y aplanamiento de las mismas
flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape) * 0.01
    for theta_shape in theta_shapes
])

Se usa minimize para optimizar la funcion encontrando las mejores Thetas posibles para esta red neuronal 

In [51]:
# Funcion de optimizacion pide Costo, Thetas, Shapes, X, Y y el Gradiente
result = op.minimize(
    fun = cost_function,
    x0 = flat_thetas,
    args=(theta_shapes, X, Y),
    method='L-BFGS-B',
    jac= back_propagation,
    options={'disp': True, 'maxiter': 3000}
    
)

/home/gustavo/Desktop/Inteligencia/RedesNeuronales/Lab2/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/home/gustavo/Desktop/Inteligencia/RedesNeuronales/Lab2/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


Comprobacion de optimizacion excitosa

In [52]:
result

      fun: nan
 hess_inv: <99385x99385 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.05977143e-06,  9.64977519e-12, -1.34429853e-10, ...,
        4.12338655e-07,  1.39474466e-08,  9.18838184e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 13166
      nit: 1891
   status: 0
  success: True
        x: array([ 2.61400345e+00,  2.68167751e-03, -5.65938327e-02, ...,
        1.06486890e+01, -4.72983009e-01, -3.45644538e+00])

In [56]:
thetasResult = result.x
thetasResult.size

99385

Se guarda las mejores Thetas en un txt para usarlas luego en el test del modelo

In [55]:
np.savetxt('thetasR.txt', thetasResult, delimiter=',') 

In [18]:
thetasResult = np.loadtxt('thetasR.txt')

# TEST

Se realizara un test para comprobar que tan bien le fue al modelo y revisar si existe algun problema de overfitting o de underfitting

In [32]:
# De igual manera se ajustan los datos para ser trabajados en la red neuronal 
X_test = X_test/1000
m,_ = X_test.shape
y_test = y_test.reshape(m,1)

Se obtienen las thetas que se calcularon previamente

In [33]:
flat_thetas_top = thetasResult

Se les da su forma correspondiente en matrices

In [34]:
superThetas = inflate_matrixes(flat_thetas_top,theta_shapes)

Ya con todos los parametros listos se realiza el feed_forward para tener la prediccion

In [35]:
a = feed_forward(superThetas, X_test)

Se obtiene la salida de la ultima capa y se modela para poder analizar la informacion

In [36]:
# Se obtienen la nuerona maxima de cada dato de test para obtener la prediccioni
maximos = np.argmax(a[-1], axis = 1).reshape(m,1)
# Se compara con los resultados verdaderos
correct = ((maximos == y_test)*1).sum()
# Se saca el porcentaje de aciertos
porBuenos = (correct/m)*100
# Y el de errores
porMalos = 100.0 - porBuenos

Se obtuvo un 86.98% de aciertos lo cual es aceptable y un valor cercano al teorico

In [37]:
porBuenos

86.98

In [38]:
porMalos

13.019999999999996

Tener un modelo que haya acertado el 86.98% de los datos lo considero màs que aceptable ya que demuestra que en efecto se entrenò de una buena manera con la capa oculta que se le asignò con 125 neuronas. Ademàs del bias que se le agregò para evitar los problemas de overfitting.